In [1]:
import sys
import pandas as pd

sys.path.append('/Users/francesco/github/LOREM-master/code')

from lorem import LOREM
from util import record2str, neuclidean, multilabel2str

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from datamanager import *

In [3]:
random_state = 0
path = '/Users/francesco/github/LOREM-master/'
path_data = path + 'dataset/'

In [3]:
# Binary

In [4]:
df, class_name = prepare_compass_dataset(path_data + 'compas-scores-two-years.csv', binary=True)
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = prepare_dataset(
    df, class_name)

In [5]:
stratify = None if isinstance(class_name, list) else df[class_name].values
X_train, X_test, Y_train, Y_test = train_test_split(df[feature_names].values, df[class_name].values, test_size=0.30,
                                                    random_state=random_state, stratify=stratify)

stratify = None if isinstance(class_name, list) else rdf[class_name].values
_, K, _, _ = train_test_split(rdf[real_feature_names].values, rdf[class_name].values, test_size=0.30,
                              random_state=random_state, stratify=stratify)

In [6]:
bb = RandomForestClassifier(n_estimators=10, random_state=random_state)
# bb = SVC(random_state=random_state)
# bb = MLPClassifier(random_state=random_state)
bb.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [7]:
i2e = 1
x = X_test[i2e]

print('x = %s' % record2str(x, feature_names, numeric_columns))
print('')

x = { age = 36, priors_count = 0, days_b_screening_arrest = 1, is_recid = 1, is_violent_recid = 0, two_year_recid = 1, length_of_stay = 0, age_cat = 25 - 45, sex = Male, race = Caucasian, c_charge_degree = F }



In [9]:
def bb_predict(X):
    return bb.predict(X)

In [10]:
bb_outcome = bb_predict(x.reshape(1, -1))[0]
bb_outcome_str = class_values[bb_outcome] if isinstance(class_name, str) else multilabel2str(bb_outcome, class_values)
print('bb(x) = { %s }' % bb_outcome_str)
print('')

bb(x) = { Medium-Low }



In [15]:
explainer = LOREM(K, bb_predict, feature_names, class_name, class_values, numeric_columns, features_map,
                      neigh_type='rndgen', categorical_use_prob=True,
                      continuous_fun_estimation=False, size=1000, ocr=0.1, multi_label=False, one_vs_rest=False,
                      random_state=random_state, verbose=True, ngen=10)
exp = explainer.explain_instance(x, samples=1000, use_weights=True, metric=neuclidean)

print('e = {\n\tr = %s\n\tc = %s    \n}' % (exp.rstr(), exp.cstr()))

calculating feature values
generating neighborhood - rndgen
gen	nevals	avg     	min     	max     
0  	250   	0.496654	0.496654	0.496654
1  	207   	0.727279	0.304293	0.993301
2  	212   	0.913403	0.253731	0.993301
3  	218   	0.923106	0.292736	0.993301
4  	199   	0.95827 	0.252717	0.993301
5  	212   	0.952931	0.361007	0.993301
6  	213   	0.96291 	0.27184 	0.993301
7  	216   	0.951381	0.269751	0.993301
8  	220   	0.947389	0.265351	0.993301
9  	215   	0.964383	0.321075	0.993301
10 	210   	0.958677	0.255155	0.993301
gen	nevals	avg	min	max
0  	250   	0.5	0.5	0.5
1  	217   	0.52866	0.303662	0.992956
2  	210   	0.567273	0.255032	0.993295
3  	213   	0.631412	0.287683	0.993295
4  	205   	0.688717	0.255816	0.993295
5  	220   	0.730914	0.253869	0.993295
6  	216   	0.75251 	0.2696  	0.993295
7  	212   	0.784349	0.253316	0.993295
8  	208   	0.779869	0.255265	0.993295
9  	219   	0.747214	0.262386	0.993295
10 	211   	0.755441	0.279334	0.993295
synthetic neighborhood class counts {'High': 412, 'Medium-L

In [26]:
Z = explainer.neighgen_fn(x, 1000)

gen	nevals	avg     	min     	max     
0  	250   	0.496654	0.496654	0.496654
1  	192   	0.692728	0.259176	0.993301
2  	216   	0.892244	0.218935	0.993301
3  	218   	0.943404	0.257961	0.993301
4  	211   	0.959468	0.315392	0.993301
5  	216   	0.961777	0.262992	0.993301
6  	217   	0.963441	0.426696	0.993301
7  	217   	0.947049	0.367896	0.993301
8  	205   	0.956602	0.346191	0.993301
9  	215   	0.955508	0.286613	0.993301
10 	228   	0.928758	0.320587	0.993301
gen	nevals	avg	min	max
0  	250   	0.5	0.5	0.5
1  	208   	0.50866	0.253231	0.992743
2  	198   	0.562578	0.253846	0.992869
3  	221   	0.62512 	0.255807	0.992743
4  	207   	0.673849	0.206469	0.993032
5  	216   	0.731685	0.256618	0.993032
6  	211   	0.784142	0.259423	0.993034
7  	208   	0.782323	0.267924	0.993034
8  	223   	0.785962	0.263436	0.993032
9  	220   	0.811258	0.292765	0.993032
10 	204   	0.791095	0.274715	0.993032


In [11]:
# Multiclass

In [12]:
df, class_name = prepare_iris_dataset(path_data + 'iris.csv')
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = prepare_dataset(
    df, class_name)

In [13]:
stratify = None if isinstance(class_name, list) else df[class_name].values
X_train, X_test, Y_train, Y_test = train_test_split(df[feature_names].values, df[class_name].values, test_size=0.30,
                                                    random_state=random_state, stratify=stratify)

stratify = None if isinstance(class_name, list) else rdf[class_name].values
_, K, _, _ = train_test_split(rdf[real_feature_names].values, rdf[class_name].values, test_size=0.30,
                              random_state=random_state, stratify=stratify)

In [14]:
bb = RandomForestClassifier(n_estimators=10, random_state=random_state)
# bb = SVC(random_state=random_state)
# bb = MLPClassifier(random_state=random_state)
bb.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [15]:
i2e = 0
x = X_test[i2e]

print('x = %s' % record2str(x, feature_names, numeric_columns))
print('')

x = { sepal length = 6.3, sepal width = 3.4, petal length = 5.6, petal width = 2.4 }



In [16]:
def bb_predict(X):
    return bb.predict(X)

In [17]:
bb_outcome = bb_predict(x.reshape(1, -1))[0]
bb_outcome_str = class_values[bb_outcome] if isinstance(class_name, str) else multilabel2str(bb_outcome, class_values)
print('bb(x) = { %s }' % bb_outcome_str)
print('')

bb(x) = { Iris-virginica }



In [18]:
explainer = LOREM(K, bb_predict, feature_names, class_name, class_values, numeric_columns, features_map,
                      neigh_type='rndgen', categorical_use_prob=True,
                      continuous_fun_estimation=False, size=1000, ocr=0.1, multi_label=False, one_vs_rest=False,
                      random_state=random_state, verbose=True, ngen=10)
exp = explainer.explain_instance(x, num_samples=1000, use_weights=True, metric=neuclidean)

print('e = {\n\tr = %s\n\tc = %s    \n}' % (exp.rstr(), exp.cstr()))

calculating feature values
generating neighborhood - rndgen
synthetic neighborhood class counts {'Iris-setosa': 190, 'Iris-versicolor': 546, 'Iris-virginica': 313}
learning local decision tree
retrieving explanation
e = {
	r = { petal length > 5.11, petal width > 0.78 } --> { class: Iris-virginica }
	c = { { petal length <= 4.62 } --> { class: Iris-versicolor }, { petal width <= 0.74 } --> { class: Iris-setosa } }    
}


In [20]:
explainer.neigh_type

'rndgen'

In [17]:
dir(explainer)

['K',
 '_LOREM__explain_tabular_instance_multiple_tree',
 '_LOREM__explain_tabular_instance_single_tree',
 '_LOREM__init_neighbor_fn',
 '__calculate_weights__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bb_predict',
 'check_feasibility',
 'class_name',
 'class_values',
 'explain_instance',
 'explain_instance_stable',
 'feature_names',
 'feature_values',
 'features_map',
 'features_map_inv',
 'filter_crules',
 'kernel',
 'kernel_width',
 'multi_label',
 'neigh_type',
 'neighgen_fn',
 'numeric_columns',
 'one_vs_rest',
 'random_state',
 'set_unfeasibible_features',
 'unadmittible_features',
 'verbose']

In [19]:
# Multilabel

In [20]:
df, class_name = prepare_yeast_dataset(path_data + 'yeast.arff')
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = prepare_dataset(
    df, class_name)

In [21]:
stratify = None if isinstance(class_name, list) else df[class_name].values
X_train, X_test, Y_train, Y_test = train_test_split(df[feature_names].values, df[class_name].values, test_size=0.30,
                                                    random_state=random_state, stratify=stratify)

stratify = None if isinstance(class_name, list) else rdf[class_name].values
_, K, _, _ = train_test_split(rdf[real_feature_names].values, rdf[class_name].values, test_size=0.30,
                              random_state=random_state, stratify=stratify)

In [22]:
bb = RandomForestClassifier(n_estimators=10, random_state=random_state)
# bb = SVC(random_state=random_state)
# bb = MLPClassifier(random_state=random_state)
bb.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [23]:
i2e = 0
x = X_test[i2e]

print('x = %s' % record2str(x, feature_names, numeric_columns))
print('')

x = { Att1 = -0.072579, Att2 = -0.090852, Att3 = -0.158701, Att4 = -0.16486, Att5 = -0.106735, Att6 = -0.102631, Att7 = 0.058959, Att8 = 0.050692, Att9 = 0.147338, Att10 = 0.116292, Att11 = 0.002095, Att12 = 0.001357, Att13 = -0.079275, Att14 = -0.070555, Att15 = -0.050414, Att16 = 0.015119, Att17 = 0.041737, Att18 = 0.108965, Att19 = -0.038776, Att20 = 0.091967, Att21 = 0.031552, Att22 = 0.180726, Att23 = 0.096652, Att24 = 0.041474, Att25 = 0.196346, Att26 = 0.057603, Att27 = 0.107898, Att28 = 0.176917, Att29 = 0.127861, Att30 = 0.062869, Att31 = 0.103735, Att32 = 0.196043, Att33 = 0.001996, Att34 = -0.170185, Att35 = -0.217192, Att36 = 0.005996, Att37 = 0.123599, Att38 = 0.16439, Att39 = 0.012365, Att40 = -0.086124, Att41 = -0.082026, Att42 = -0.014118, Att43 = 0.081171, Att44 = 0.034226, Att45 = -0.046256, Att46 = -0.080571, Att47 = -0.066226, Att48 = 0.052526, Att49 = 0.062243, Att50 = -0.001041, Att51 = -0.057956, Att52 = 0.054652, Att53 = 0.130336, Att54 = 0.082482, Att55 = -0.05

In [24]:
def bb_predict(X):
    return bb.predict(X)

In [25]:
bb_outcome = bb_predict(x.reshape(1, -1))[0]
bb_outcome_str = class_values[bb_outcome] if isinstance(class_name, str) else multilabel2str(bb_outcome, class_values)
print('bb(x) = { %s }' % bb_outcome_str)
print('')

bb(x) = { Class7, Class8 }



In [26]:
explainer = LOREM(K, bb_predict, feature_names, class_name, class_values, numeric_columns, features_map,
                  neigh_type='closest', categorical_use_prob=True,
                  continuous_fun_estimation=False, size=1000, ocr=0.1, multi_label=True, one_vs_rest=False,
                  random_state=random_state, verbose=True, Kc=X_test, core_neigh_type='unified')
exp = explainer.explain_instance(x, num_samples=1000, use_weights=True, metric=neuclidean)

print('e = {\n\tr = %s\n\tc = %s    \n}' % (exp.rstr(), exp.cstr()))
# for crule in exp.crules:
#     print(crule)
print(exp.bb_pred, exp.dt_pred, exp.fidelity)

generating neighborhood - closest
calculating feature values
synthetic neighborhood class counts {'Class1': 85.0, 'Class10': 275.0, 'Class11': 235.0, 'Class12': 79.0, 'Class13': 24.0, 'Class14': 21.0, 'Class2': 5.0, 'Class3': 5.0, 'Class4': 0.0, 'Class5': 0.0, 'Class6': 3.0, 'Class7': 846.0, 'Class8': 831.0, 'Class9': 0.0}
learning local decision tree
retrieving explanation
e = {
	r = { Att88 > -0.03, Att37 <= 0.20, Att72 <= 0.02, Att6 <= 0.10, Att24 <= 0.12, Att32 <= 0.29, Att59 > -0.21, Att93 <= 0.43, Att17 <= 0.17, Att102 > -0.01, Att99 <= 0.02, Att56 <= 0.25, Att9 <= 0.22, Att92 > -0.26, Att25 <= 0.22, Att25 > -0.20, Att27 <= 0.22, Att90 > -0.18, Att94 > -0.12, Att30 > -0.20, Att74 <= 0.17, Att85 <= 0.19, Att64 <= 0.26, Att49 > -0.27, Att101 <= 0.06, Att33 > -0.11, Att70 <= 0.18, Att8 <= 0.08, Att5 > -0.26, Att51 > -0.18, Att34 > -0.21, Att15 > -0.15 } --> { Class7, Class8 }
	c = { { Att27 > 0.22 } --> { Class13, Class14, Class7, Class8 }, { Att70 > 0.18 } --> { Class10, Class7, Cl